In [31]:
import numpy as np
import pandas as pd

# Load the data
data_df = pd.read_excel('../datasets/UK_Online_Retail/Online Retail.xlsx')

In [32]:
data_df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [33]:
# calculate total price
data_df['TotalPrice'] = data_df['Quantity'] * data_df['UnitPrice']


# group by CustomerID
grouped_df = data_df.groupby(['CustomerID']).agg({
    'InvoiceNo': 'count',
    'Quantity': 'sum',
    'UnitPrice': 'sum',
    'TotalPrice': 'sum',
    'InvoiceDate': 'max',
    'Country': 'first'
})

# rename columns
grouped_df.rename(columns={
    'InvoiceNo': 'Frequency',
    'Quantity': 'TotalQuantity',
    'UnitPrice': 'TotalUnitPrice',
    'TotalPrice': 'MonetaryValue',
    'InvoiceDate': 'LastPurchaseDate'
}, inplace=True)

In [34]:
# create a snapshot date for recency calculation
snapshot_date = grouped_df['LastPurchaseDate'].max() + pd.DateOffset(days=1)


# calculate recency, frequency and monetary value
grouped_df['Recency'] = (snapshot_date - grouped_df['LastPurchaseDate']).dt.days
grouped_df['Frequency'] = grouped_df['Frequency'].astype(np.int32)
grouped_df['Recency'] = grouped_df['Recency'].astype(np.int64).astype(np.int32)
grouped_df['MonetaryValue'] = grouped_df['MonetaryValue'].astype(np.float32)
grouped_df.reset_index(inplace=True)
grouped_df.head()

,CustomerID,Frequency,TotalQuantity,TotalUnitPrice,MonetaryValue,LastPurchaseDate,Country,Recency
0,12346.0,2,0,2.08,0.000000,2011-01-18 10:17:00,United Kingdom,326
1,12347.0,182,2458,481.21,4310.000000,2011-12-07 15:52:00,Iceland,2
2,12348.0,31,2341,178.71,1797.239990,2011-09-25 13:13:00,Finland,75
3,12349.0,73,631,605.10,1757.550049,2011-11-21 09:51:00,Italy,19
4,12350.0,17,197,65.30,334.399994,2011-02-02 16:01:00,Norway,310


In [35]:
# leave only the columns we need 
rfm_df = grouped_df[['CustomerID', 'Recency', 'Frequency', 'MonetaryValue', 'Country']]

In [36]:
def rfm_scores(df):
    rfm = df.copy()
    recency_scores = [5, 4, 3, 2, 1]
    frequency_scores = [1, 2, 3, 4, 5]
    monetary_scores = [1, 2, 3, 4, 5]
    
    rfm['RecencyScore'] = pd.qcut(rfm['Recency'], 5, labels=recency_scores)
    rfm['FrequencyScore'] = pd.qcut(rfm['Frequency'], 5, labels=frequency_scores)
    rfm['MonetaryScore'] = pd.qcut(rfm['MonetaryValue'], 5, labels=monetary_scores)
    
    # Convert RFM scores to numeric type
    rfm['RecencyScore'] = rfm['RecencyScore'].astype(float)
    rfm['FrequencyScore'] = rfm['FrequencyScore'].astype(float)
    rfm['MonetaryScore'] = rfm['MonetaryScore'].astype(float)
    
    rfm['RecencyScoreText'] = rfm['RecencyScore'].apply(lambda x: 
        'Very recently' if x == 5 
        else 'recently' if x == 4 
        else 'no so recently' if x == 3 
        else 'some time ago' if x == 2 
        else 'a very long time ago')
    rfm['FrequencyScoreText'] = rfm['FrequencyScore'].apply(lambda x: 
        'Very frequently' if x == 5 
        else 'Frequently' if x == 4 
        else 'Average' if x == 3 
        else 'Not so frequently' if x == 2 
        else 'Not frequently at all')
    rfm['MonetaryScoreText'] = rfm['MonetaryScore'].apply(lambda x: 'Very high' if x == 5 else 'High' if x == 4 else 'Average' if x == 3 else 'Low' if x == 2 else 'Very low')

    return rfm

In [37]:
rfm_df = rfm_scores(rfm_df)

In [39]:
rfm_df

,CustomerID,Recency,Frequency,MonetaryValue,Country,RecencyScore,FrequencyScore,MonetaryScore,RecencyScoreText,FrequencyScoreText,MonetaryScoreText
0,12346.0,326,2,0.000000,United Kingdom,1.0,1.0,1.0,Not recently at all,Not frequently at all,Very low
1,12347.0,2,182,4310.000000,Iceland,5.0,5.0,5.0,Very recently,Very frequently,Very high
2,12348.0,75,31,1797.239990,Finland,2.0,3.0,4.0,Not so recently,Average,High
3,12349.0,19,73,1757.550049,Italy,4.0,4.0,4.0,Recently,Frequently,High
4,12350.0,310,17,334.399994,Norway,1.0,2.0,2.0,Not recently at all,Not so frequently,Low
...,...,...,...,...,...,...,...,...,...,...,...
4367,18280.0,278,10,180.600006,United Kingdom,1.0,1.0,1.0,Not recently at all,Not frequently at all,Very low
4368,18281.0,181,7,80.820000,United Kingdom,1.0,1.0,1.0,Not recently at all,Not frequently at all,Very low
4369,18282.0,8,13,176.600006,United Kingdom,5.0,1.0,1.0,Very recently,Not frequently at all,Very low
4370,18283.0,4,756,2094.879883,United Kingdom,5.0,5.0,5.0,Very recently,Very frequently,Very high


In [29]:
# save the data
rfm_df.to_csv('../datasets/UK_Online_Retail/rfm_and_country.csv')

In [30]:
from sentence_transformers import SentenceTransformer
cust_id_df = []
def compile_text(x):
    cust_id_df.append(x['CustomerID'])
    # print (x)
    
    # individual_reviews = x['reviews'].split(';;')
    
    
    text =  f"""Recency: {x['RecencyScoreText']},
                Frequency: {x['FrequencyScoreText']},
                Monetary Value: {x['MonetaryScoreText']},
                Country: {x['Country']}
            """
    # text =  f"""Reviews: {x['reviews']},
    #             Average Review Score: {int(x['avg_review_score'])}
    #         """
    print(text)
    return text
# def compile_text(x):
#     cust_id_df.append(x['customer_id'])
#     # print (x)
#     text =  f"""Recency: {x['recency']},
#                 Frequency: {x['frequency']},
#                 Monetary Value: {x['monetary_value']},
#                 Average Review Score: {x['avg_review_score']},
#             """
#     individual_reviews = x['reviews'].split(';;')

#     for review in individual_reviews:
#         text += f"""Review: {review},\n"""    
#     # text =  f"""Reviews: {x['reviews']},
#     #             Average Review Score: {int(x['avg_review_score'])}
#     #         """
#     print(text)
#     return text
sentences = rfm_df.apply(lambda x: compile_text(x), axis=1).tolist()

model = SentenceTransformer(r"sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

output = model.encode(sentences=sentences, show_progress_bar= True, normalize_embeddings  = True)

cust_id_df = pd.DataFrame(cust_id_df, columns=['customer_id'])

# Join the embeddings with the respective customer_id (customer id is a string list)
output = np.concatenate((cust_id_df, output), axis=1)

df_embedding = pd.DataFrame(output)

print(df_embedding)

df_embedding.to_csv('../datasets/UK_Online_Retail/rfm_and_country_embedding.csv', index=False)

Recency: Not recently at all,
                Frequency: Not frequently at all,
                Monetary Value: Very low,
                Country: United Kingdom
            
Recency: Very recently,
                Frequency: Very frequently,
                Monetary Value: Very high,
                Country: Iceland
            
Recency: Not so recently,
                Frequency: Average,
                Monetary Value: High,
                Country: Finland
            
Recency: Recently,
                Frequency: Frequently,
                Monetary Value: High,
                Country: Italy
            
Recency: Not recently at all,
                Frequency: Not so frequently,
                Monetary Value: Low,
                Country: Norway
            
Recency: Average,
                Frequency: Frequently,
                Monetary Value: High,
                Country: Norway
            
Recency: Not recently at all,
                Frequency: Not frequently at all,
    

Batches: 100%|██████████| 137/137 [00:05<00:00, 24.72it/s]


          0         1         2         3         4         5         6    \
0     12346.0 -0.043996 -0.028806 -0.004085 -0.013536 -0.011616  0.024026   
1     12347.0 -0.006102 -0.075146 -0.003075  0.001080  0.009851 -0.010879   
2     12348.0 -0.038025 -0.047315 -0.002999 -0.031102  0.007563 -0.002664   
3     12349.0 -0.045898 -0.014613 -0.002815 -0.005218  0.010050  0.012821   
4     12350.0 -0.016405 -0.014284 -0.003333 -0.004498 -0.020842 -0.012472   
...       ...       ...       ...       ...       ...       ...       ...   
4367  18280.0 -0.043996 -0.028806 -0.004085 -0.013536 -0.011616  0.024026   
4368  18281.0 -0.043996 -0.028806 -0.004085 -0.013536 -0.011616  0.024026   
4369  18282.0 -0.030680 -0.041915 -0.003772 -0.020092 -0.006186  0.019327   
4370  18283.0 -0.023302 -0.065546 -0.003783 -0.018509  0.000860  0.022903   
4371  18287.0 -0.056201 -0.066306 -0.003715 -0.023690 -0.002184  0.043805   

           7         8         9    ...       759       760       761  \
0 

In [34]:
import sys  
sys.exit(0)

SystemExit: 0

C:\Users\Shamika Tissera\AppData\Roaming\Python\Python310\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
order_df.loc[order_df['order_id'] == '00010242fe8c5a6d1ba2dd792cb16214']

In [ ]:
order_items_df.loc[order_items_df['order_id'] == 'ca3625898fbd48669d50701aba51cd5f']

In [ ]:
# # check if all the rows having the same order_id has the same product_id
# order_items_df.groupby('order_id').agg({'product_id': 'nunique'}).sort_values(by='product_id', ascending=False)


In [ ]:
### Calculation of the total order value after grouping by order_id

order_items_df = order_items_df.groupby(['order_id']).agg({
    'order_item_id': 'count',
    'price': 'sum',
    'freight_value': 'sum',
    'shipping_limit_date': 'max'
}).reset_index()

order_items_df.rename(columns={
    'order_item_id': 'order_item_count',
    'price': 'order_value',
    'shipping_limit_date': 'shipping_limit_date_max'
}, inplace=True)

order_items_df['total_order_value'] = order_items_df['order_value'] + order_items_df['freight_value']

# order_items_df.loc[order_items_df['order_id'] == '00143d0f86d6fbd9f9b38ab440ac16f5']


In [ ]:
order_items_df.loc[order_items_df['order_id'] == 'e481f51cbdc54678b7cc49136f2d6af7']

In [ ]:
order_items_df

In [ ]:
print(order_payment_df.describe)
print(order_payment_df.nunique())

In [ ]:
print(order_payment_df[order_payment_df['order_id'].duplicated()])

In [ ]:
order_payment_df.loc[order_payment_df['order_id'] == 'c9b01bef18eb84888f0fd071b8413b38']

In [ ]:
order_df.describe

In [ ]:
order_items_df.describe

In [ ]:
merged_df = pd.merge(reviews_df, order_df, on='order_id', how='inner', validate='many_to_one')


In [ ]:
merged_df = pd.merge(merged_df, order_payment_df, on='order_id')

In [ ]:
merged_df = pd.merge(merged_df, order_items_df, on='order_id')

In [ ]:
merged_df.nunique()

In [ ]:
# # Merge the dataframes
# merged_df = pd.merge(merged_df, order_payment_df, on='order_id')
# merged_df = pd.merge(merged_df, order_items_df, on='order_id')
# # merged_df
# # merged_df = pd.merge(merged_df, products_df, on='product_id')
# # merged_df = pd.merge(merged_df, product_types_df, on='product_category_name')


# # Drop unnecessary columns
# # merged_df = merged_df.drop(['order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date', 'payment_sequential', 'payment_type', 'payment_installments', 'order_item_id', 'seller_id', 'shipping_limit_date', 'product_name_lenght', 'product_description_lenght', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm', 'product_category_name', 'product_category_name_english'], axis=1)

# merged_df.columns

In [ ]:
merged_df_without_null_comments = merged_df[merged_df['review_comment_message'].notna()] # Remove rows with null values in review_comment_message

In [ ]:
# calculate the RFM values
merged_df_without_null_comments['order_purchase_timestamp'] = pd.to_datetime(merged_df_without_null_comments['order_purchase_timestamp'])
merged_df_without_null_comments['order_purchase_timestamp'].min()

# Create a hypothetical snapshot_day data as if we're doing analysis recently
snapshot_date = merged_df_without_null_comments['order_purchase_timestamp'].max() + pd.DateOffset(days=1)

# Aggregate data on a customer level
datamart = merged_df_without_null_comments.groupby(['customer_id']).agg({
    'order_purchase_timestamp': lambda x: (snapshot_date - x.max()).days,
    'order_id': 'count',
    'total_order_value': 'sum',
    'review_score': 'mean',
    'review_comment_message': lambda x: ';;'.join(map(str, x))
}).reset_index()

# Rename the columns
datamart.rename(columns={
    'order_purchase_timestamp': 'recency',
    'order_id': 'frequency',
    'total_order_value': 'monetary_value',
    'review_score': 'avg_review_score',
    'review_comment_message': 'reviews'
}, inplace=True)
datamart

In [ ]:
datamart['reviews'].notna().sum()

In [ ]:
# # Aggregate data on a customer level
# datamart = merged_df.groupby(['customer_id']).agg({
#     'order_purchase_timestamp': lambda x: (snapshot_date - pd.to_datetime(x.max())).days,
#     'order_id': 'count',
#     'total_order_value': 'sum',
#     'review_score': 'mean',
#     'review_comment_message': lambda x: ','.join(map(str, x))
# }).reset_index()

In [ ]:
def rfm_scores(df):
    rfm = df.copy()
    recency_scores = [5, 4, 3, 2, 1]
    frequency_scores = [1, 2, 3, 4, 5]
    monetary_scores = [1, 2, 3, 4, 5]
    
    rfm['RecencyScore'] = pd.qcut(rfm['recency'], 5, labels=recency_scores)
    rfm['FrequencyScore'] = pd.qcut(rfm['frequency'], 5, labels=frequency_scores)
    rfm['MonetaryScore'] = pd.qcut(rfm['monetary_value'], 5, labels=monetary_scores)
    
    # Convert RFM scores to numeric type
    rfm['RecencyScore'] = rfm['RecencyScore'].astype(float)
    rfm['FrequencyScore'] = rfm['FrequencyScore'].astype(float)
    rfm['MonetaryScore'] = rfm['MonetaryScore'].astype(float)
    
    rfm['RecencyScoreText'] = rfm['RecencyScore'].apply(lambda x: 'Very recently' if x == 5 else 'Recently' if x == 4 else 'Average' if x == 3 else 'Not so recently' if x == 2 else 'Not recently at all')
    rfm['FrequencyScoreText'] = rfm['FrequencyScore'].apply(lambda x: 'Very frequently' if x == 5 else 'Frequently' if x == 4 else 'Average' if x == 3 else 'Not so frequently' if x == 2 else 'Not frequently at all')
    rfm['MonetaryScoreText'] = rfm['MonetaryScore'].apply(lambda x: 'Very high' if x == 5 else 'High' if x == 4 else 'Average' if x == 3 else 'Low' if x == 2 else 'Very low')

    return rfm

In [ ]:
datamart = rfm_scores(datamart)

In [ ]:
datamart.to_csv('datasets/rfm.csv', index=False)

In [ ]:
from sentence_transformers import SentenceTransformer
cust_id_df = []
def compile_text(x):
    cust_id_df.append(x['customer_id'])
    # print (x)
    
    individual_reviews = x['reviews'].split(';;')
    
    
    text =  f"""Recency: {x['recency']},
                Frequency: {x['frequency']},
                Monetary Value: {x['monetary_value']},
            """
    # text =  f"""Reviews: {x['reviews']},
    #             Average Review Score: {int(x['avg_review_score'])}
    #         """
    print(text)
    return text
# def compile_text(x):
#     cust_id_df.append(x['customer_id'])
#     # print (x)
#     text =  f"""Recency: {x['recency']},
#                 Frequency: {x['frequency']},
#                 Monetary Value: {x['monetary_value']},
#                 Average Review Score: {x['avg_review_score']},
#             """
#     individual_reviews = x['reviews'].split(';;')

#     for review in individual_reviews:
#         text += f"""Review: {review},\n"""    
#     # text =  f"""Reviews: {x['reviews']},
#     #             Average Review Score: {int(x['avg_review_score'])}
#     #         """
#     print(text)
#     return text
sentences = datamart.apply(lambda x: compile_text(x), axis=1).tolist()

model = SentenceTransformer(r"sentence-transformers/paraphrase-MiniLM-L6-v2")

output = model.encode(sentences=sentences, show_progress_bar= True, normalize_embeddings  = True)

cust_id_df = pd.DataFrame(cust_id_df, columns=['customer_id'])

# Join the embeddings with the respective customer_id (customer id is a string list)
output = np.concatenate((cust_id_df, output), axis=1)

df_embedding = pd.DataFrame(output)

print(df_embedding)

df_embedding.to_csv('datasets/embeddings.csv', index=False)

In [ ]:
cust_id_df

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

torch.cuda.empty_cache()

#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

def compile_text(x):
    text =  f"""Recency: {x['Recency']},  
                Frequency: {x['Frequency']}, 
                Monetary Value: {x['MonetaryValue']}
            """
    print(text)
    return text

sentences = datamart.head(1000).apply(lambda x: compile_text(x), axis=1).tolist()

# Load model from HuggingFace Hub
from torch.cuda.amp import GradScaler, autocast

# Compute sentence embeddings
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
model = AutoModel.from_pretrained("sentence-transformers/bert-base-nli-mean-tokens")
model.to('cuda')

sentence_embeddings = []

for sentence in sentences:
    input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0).to('cuda')  # Batch size 1
    attention_mask = input_ids.ne(0).to('cuda')  # Batch size 1
    with torch.no_grad():
        model_output = model(input_ids, attention_mask=attention_mask)
    sentence_embedding = mean_pooling(model_output, attention_mask)
    sentence_embeddings.append(sentence_embedding.cpu().numpy())

sentence_embeddings = np.concatenate(sentence_embeddings, axis=0)

# Compute cosine similarity between all pairs
cosine_similarities = cosine_similarity(csr_matrix(sentence_embeddings).reshape(-1, 1))

# Convert to pandas dataframe
cosine_similarities_df = pd.DataFrame(cosine_similarities, columns=datamart.head(1000)['customer_id'], index=datamart.head(1000)['customer_id'])

cosine_similarities_df.to_csv('datasets/cosine_similarities.csv')
print(cosine_similarities_df)

# Compute cosine similarity between all pairs
cosine_similarities = cosine_similarity(sentence_embeddings)

# Convert to pandas dataframe
cosine_similarities_df = pd.DataFrame(cosine_similarities, columns=datamart['customer_id'], index=datamart['customer_id'])

cosine_similarities_df.to_csv('datasets/cosine_similarities.csv')
print(cosine_similarities_df)

In [ ]:
sentences = df.apply(lambda x: compile_text(x), axis=1).tolist()



model = SentenceTransformer(r"sentence-transformers/paraphrase-MiniLM-L6-v2")

output = model.encode(sentences=sentences, show_progress_bar= True, normalize_embeddings  = True)

df_embedding = pd.DataFrame(output)
df_embedding


df_embedding.to_csv("embedding_train.csv",index = False)

In [ ]:
merged_df.nunique()

In [ ]:
merged_df

In [ ]:
datamart_df

In [ ]:
# number of unique values in each column
print(datamart_df.nunique())

Addition of the 'V' and 'C' values as used in Wu, Z., Jin, L., Zhao, J., Jing, L., & Chen, L. (2022). Research on Segmenting E-Commerce Customer through an Improved K-Medoids Clustering Algorithm. Computational Intelligence and Neuroscience, 2022, 1–10. https://doi.org/10.1155/2022/9930613


In [ ]:
order_items_df.loc[order_items_df['order_id'] == '00143d0f86d6fbd9f9b38ab440ac16f5']
